In [1]:
from itertools import product
import torch

In [8]:
w, h = 3, 3

ls = [[0, 1] for _ in range(w*h)]

In [17]:
indicator_dataset = None

for idx, i in enumerate(product(*ls)):
    arr = torch.tensor(list(i)).reshape(1, 3, 3)
    if indicator_dataset is None:
        indicator_dataset = arr
    else:
        indicator_dataset = torch.concatenate([indicator_dataset, arr])

print(indicator_dataset.shape)

torch.Size([512, 3, 3])


In [23]:
stack_arr = torch.stack([indicator_dataset, indicator_dataset], dim=1)

In [37]:
# filter out cells which is confident [0, epsilon] OR [1-epsilon, 1]

epsilon = 0.1

stack_arr[(stack_arr[:, 0, 2, 2] < epsilon) + (stack_arr[:, 0, 2, 2] > 1-epsilon)].shape

torch.Size([512, 2, 3, 3])

In [ ]:
net: torch.nn.Module

out_ = net(indicator_dataset) # out shape: [512, 2, 3, 3]
# calculate probability of 0
out_ = torch.softmax(out_, dim=1)[:, 1, ...] # out shape: [512, 1, 3, 3]

stack_arr = torch.stack([indicator_dataset, out_], dim=1)

# 过滤输出 confidence 足够高的，过滤出高置信度对
filtered_arr = stack_arr[(stack_arr[:, 0, 2, 2] < epsilon) + (stack_arr[:, 0, 2, 2] > 1-epsilon)]

# 做用旋转不变性做商集
filtered_arr[:, 0, ...]